# W281 Final Project: Intel Image Classification Model #

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from collections import Counter
import gc

from PIL import Image
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,BatchNormalization,MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
!pwd

/Users/zdbrown13/w281/w281_Final_Project_Brown_Benzoni_Olaya


In [3]:
def load_images(directory, num_files=5):
    images = []
    file_list = os.listdir(directory)[:num_files]  # Load only the first 5 files

    for filename in tqdm(file_list, desc=f"Loading images from {directory}"):
        img_path = os.path.join(directory, filename)
        img = Image.open(img_path)
        img = img.resize((150, 150))  # Resize image to 150 x 150
        images.append(img)

    return images

In [4]:
# Image Paths

buildings_train = './seg_train/buildings'
forest_train = './seg_train/forest'
glacier_train = './seg_train/glacier'
mountain_train = './seg_train/mountain'
sea_train = './seg_train/sea'
street_train = './seg_train/street'

buildings_test = './seg_test/buildings'
forest_test = './seg_test/forest'
glacier_test = './seg_test/glacier'
mountain_test = './seg_test/mountain'
sea_test = './seg_test/sea'
street_test = './seg_test/street'

In [5]:
# Load images from each category
buildings_img = load_images(buildings_train)
forest_img = load_images(forest_train)
glacier_img = load_images(glacier_train)
mountain_img = load_images(mountain_train)
sea_img = load_images(sea_train)
street_img = load_images(street_train)

Loading images from ./seg_train/street: 100%|██████████| 5/5 [00:00<00:00, 1187.65it/s]


In [6]:
buildings_img

[<PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>,
 <PIL.Image.Image image mode=RGB size=150x150>]

## Load Data ##

In [7]:
def load_data(datasets):
    
    output = []

    for dataset in datasets:
        images, labels = [], []
        print(f"Loading {dataset}...")
        i = 0
        for folder in os.listdir(dataset):
            label = i # Converting word labels to int (i.e. buildings = 0)
            i = i+1
            folder_path = os.path.join(dataset, folder)

            for file in tqdm(os.listdir(folder_path), desc=f"Processing {folder}"):
                img_path = os.path.join(folder_path, file)

                image = Image.open(img_path).resize((150, 150))
                images.append(image)
                labels.append(label)
                
        images = np.stack(images)
        labels = np.array(labels, dtype='int32')
        output.append((images, labels))
        
    return output

In [8]:
datasets = ["/Users/zdbrown13/w281/w281_Final_Project_Brown_Benzoni_Olaya/seg_train", 
        "/Users/zdbrown13/w281/w281_Final_Project_Brown_Benzoni_Olaya/seg_test"]


train_path, test_path = datasets

In [9]:


(train_images, train_labels), (test_images, test_labels) = load_data(datasets)


Loading /Users/zdbrown13/w281/w281_Final_Project_Brown_Benzoni_Olaya/seg_train...


Processing sea: 100%|██████████| 2274/2274 [00:01<00:00, 2115.32it/s]


Loading /Users/zdbrown13/w281/w281_Final_Project_Brown_Benzoni_Olaya/seg_test...


Processing sea: 100%|██████████| 510/510 [00:00<00:00, 2146.29it/s]


In [10]:
# Normalization
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [11]:
# Training Data Distribution
unique, counts = np.unique(train_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 2271, 1: 2191, 2: 2404, 3: 2382, 4: 2512, 5: 2274}


In [12]:
avg = sum(counts)/len(unique)
print(avg)

2339.0


In [13]:
# Test Data Distribution
test_unique, test_counts = np.unique(test_labels, return_counts=True)
print(dict(zip(test_unique, test_counts)))

{0: 474, 1: 437, 2: 553, 3: 501, 4: 525, 5: 510}


In [14]:
test_avg = sum(test_counts)/len(test_unique)
print(test_avg)

500.0


## Random Forest & SVM

In [15]:
# Flatten the data for Random Forest & SVM
X_train_flat = train_images.reshape(len(train_images), -1)
X_test_flat = test_images.reshape(len(test_images), -1)
y_train_trad = train_labels
y_test_trad = test_labels

# Create train/validation splits
X_train_ml, X_val_ml, y_train_ml, y_val_ml = train_test_split(
    X_train_flat,
    y_train_trad,
    test_size=0.2,
    random_state=42
)

In [16]:
# RandomForest Model
print("Training Random Forest Classifier...")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_ml, y_train_ml)

# Evaluate Random Forest on validation set
rf_val_pred = rf.predict(X_val_ml)
print("\nRandom Forest Validation Results:")
print(classification_report(y_val_ml, rf_val_pred))
print(f"Validation Accuracy: {accuracy_score(y_val_ml, rf_val_pred):.3f}")

Training Random Forest Classifier...

Random Forest Validation Results:
              precision    recall  f1-score   support

           0       0.71      0.83      0.77       462
           1       0.53      0.38      0.45       460
           2       0.57      0.62      0.60       455
           3       0.66      0.66      0.66       484
           4       0.58      0.64      0.61       538
           5       0.49      0.45      0.46       408

    accuracy                           0.60      2807
   macro avg       0.59      0.60      0.59      2807
weighted avg       0.59      0.60      0.59      2807

Validation Accuracy: 0.601


In [ ]:
def train_svm_classifier(X_train, y_train, X_val, y_val):
    """
    Train SVM classifier with hyperparameter tuning
    """
    # Free memory before starting
    gc.collect()

    # Define parameter grid
    param_grid = {
        'C': [0.1, 1.0],  # Regularization parameter
        'kernel': ['rbf'],  # Radial Basis Function kernel
        'gamma': ['scale', 'auto']  # Kernel coefficient
    }
    
    # Initialize SVM, specifying 2GB cache size for M1 chip.
    svm = SVC(random_state=42, cache_size=2000)
    
    # Perform grid search
    print("Starting SVM grid search...")
    grid_search = GridSearchCV(
        svm,
        param_grid,
        cv=2,  # 2-fold cross-validation
        n_jobs=-1,
        verbose=2,
        scoring='accuracy'
    )
    
    # Fit the model
    grid_search.fit(X_train, y_train)
    
    # Print results
    print("\nBest parameters found:")
    print(grid_search.best_params_)
    print("\nBest cross-validation accuracy: {:.3f}".format(grid_search.best_score_))
    
    # Evaluate on validation set
    val_accuracy = grid_search.score(X_val, y_val)
    print("Validation accuracy: {:.3f}".format(val_accuracy))
    
    return grid_search.best_estimator_

print("\nTraining SVM classifier...")
svm_classifier = train_svm_classifier(X_train_ml, y_train_ml, X_val_ml, y_val_ml)

# Evaluate SVM on test set
svm_test_pred = svm_classifier.predict(X_test_flat)
print("\nSVM Test Results:")
print(classification_report(y_test_trad, svm_test_pred))
print(f"Test Accuracy: {accuracy_score(y_test_trad, svm_test_pred):.3f}")


Training SVM classifier...
Starting SVM grid search...
Fitting 2 folds for each of 4 candidates, totalling 8 fits


## CNN & ResNet

In [ ]:
# Convert labels to one-hot encoding for CNN/ResNet
num_classes = len(np.unique(train_labels))
train_labels_onehot = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels_onehot = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create train/validation split for CNN/ResNet
X_train, X_val, y_train, y_val = train_test_split(
    train_images, 
    train_labels_onehot,
    test_size=0.2, 
    random_state=42
)

In [ ]:
# Define a CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Compile the CNN model
cnn_model = create_cnn_model((150, 150, 3), num_classes)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN Model
cnn_history = cnn_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_val, y_val)
)

# Evaluate the CNN Model
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_images, test_labels_onehot)
print(f"CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")

In [ ]:
# ResNet50 Transfer Learning Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

resnet_model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the ResNet model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ResNet Model
resnet_history = resnet_model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=10,
    validation_data=(X_val, y_val)
)

# Evaluate the ResNet Model
resnet_loss, resnet_accuracy = resnet_model.evaluate(test_images, test_labels_onehot)
print(f"ResNet50 Model Accuracy: {resnet_accuracy * 100:.2f}%")

## Random Forest vs. SVM vs. CNN vs. ResNet Evaluation

In [ ]:
# Summary of Model Performances
print("\nModel Performance Summary:")
print(f"Random Forest Model Accuracy: {accuracy_score(y_test_rf, y_pred_rf) * 100:.2f}%")
print(f"CNN Model Accuracy: {cnn_accuracy * 100:.2f}%")
print(f"ResNet50 Model Accuracy: {resnet_accuracy * 100:.2f}%")